Import necessary pakages

In [1]:
using JuMP
using HiGHS

In [2]:
X_coor = [55 58 60 64 70 66 71 72 73 76 68 68 75 70]
Y_coor = [60 55 54 56 65 63 60 50 66 44 43 67 47 42]
n = length(X_coor) # number of points
k = 3 # number of clusters

3

In [3]:
d = Array{Float64}(undef, n, n)
for i=1:n
    for j=1:n
        d[i, j] = sqrt((X_coor[i] - X_coor[j])^2 + (Y_coor[i] - Y_coor[j])^2)
    end
end

In [4]:
model = Model(HiGHS.Optimizer)
@variable(model, x[i=1:n], Bin) # x is indicator variable states weather or not point i is selected as a medoid
@variable(model, y[i=1:n, j=1:n], Bin) # y is an indicator variable states weather point j is assigned to cluster of meodoid-i
@objective(model, Min, sum(d[i,j] * y[i,j] for i=1:k for j=1:n))
# There can be only 3 clusters
@constraint(model, sum(x[i] for i=1:n) == 3)
# Each point should be assigned to only one cluster
@constraint(model, [j=1:n], sum(y[i,j] for i=1:n) == 1)
# Each cluster should have one medoid point
@constraint(model, [i=1:n, j=1:n], y[i,j] <= x[i])
# If a point is assigned as a medoid of a cluster, then this point should within that cluster.
@constraint(model, [i=1:n], x[i] <= y[i,i])

print(model)
optimize!(model)

println("Termination status: $(termination_status(model))")
if termination_status(model) == MOI.OPTIMAL
    println("Optimal objective value: $(objective_value(model))")
    println("Selected medoid points:")
    for i=1:n
        if value(x[i])>0
            println("======================")
            println("New Cluster")
            println("======================")
            println("Medoid Point: $(i)")
            println("Assigned points")
            for j=1:n
                if value(y[i,j])>0
                    println("Point: $(j)")
                end
            end
        end
    end
else
    println("No optimal solution available")
end

Running HiGHS 1.9.0 (git hash: 66f735e60): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [2e+00, 3e+01]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 3e+00]
Presolving model
225 rows, 210 cols, 630 nonzeros  0s
197 rows, 196 cols, 574 nonzeros  0s

Solving MIP model with:
   197 rows
   196 cols (196 binary, 0 integer, 0 implied int., 0 continuous)
   574 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               in